In [1]:
import os
import random
from Mydataset import MYDataset
import math
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import r2_score
from IPython.display import clear_output as clear
import statsmodels.api as sm
from SWNN import SWNN

In [6]:
def select(data, ratio):
    year = 0
    month = 0
    tdf = pd.DataFrame(columns=data.columns)
    positive = pd.DataFrame(columns=data.columns)
    nagative = pd.DataFrame(columns=data.columns)
    index = -1
    data.sort_values(by=['year', 'month'])
    for i in tqdm(range(0,data.shape[0])):
        if not year == data.loc[i]['year'] or not month == data.loc[i]['month']:
            if not index == -1:
                tp = tdf.sample(frac=ratio)
                tn = tdf.append(tp)
                tn.drop_duplicates(keep=False, inplace=True)
                # print(tdf)
                # print(tp)
                # print(tn)
                positive = positive.append(tp)
                nagative = nagative.append(tn)
            tdf = pd.DataFrame(columns=data.columns)
            year = data.loc[i]['year']
            month = data.loc[i]['month']
            index = i
        tdf.loc[i-index] = data.loc[i]
    return positive, nagative

varName = ['fCO2', 'Chl', 'Temp', 'Salt']
varNum = len(varName)
batch_size = 256

dataset = pd.read_csv("D://CO2_data5.csv", encoding="utf-8")
dataset = dataset.dropna()
dataset = dataset[dataset.index % 4 == 0]

df0 = dataset['date'].str.split("/",expand = True)
df0.columns = ['year', 'month', 'date']

dataset['month'] = df0['month']
dataset['year'] = df0['year']

dataset = dataset.reset_index()

test_set, train_set = select(dataset, 0.2)


mean_li = []
std_li = []

for i in range(0, varNum, 1):
    mean_li.append(train_set[varName[i]].mean())
    std_li.append(train_set[varName[i]].std())

train_set = train_set.copy()
test_set = test_set.copy()

for i in range(0, varNum, 1):
    train_set.loc[:, varName[i]] = (train_set[varName[i]].copy() - mean_li[i] + 1.0) / std_li[i]
    test_set.loc[:, varName[i]] = (test_set[varName[i]].copy() - mean_li[i] + 1.0) / std_li[i]

del mean_li, std_li


  0%|          | 279/85506 [00:01<04:27, 318.78it/s]D:\Temp\ipykernel_9160\4172117263.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tn = tdf.append(tp)
D:\Temp\ipykernel_9160\4172117263.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  positive = positive.append(tp)
D:\Temp\ipykernel_9160\4172117263.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nagative = nagative.append(tn)
  1%|          | 591/85506 [00:02<05:15, 269.28it/s]D:\Temp\ipykernel_9160\4172117263.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tn = tdf.append(tp)
D:\Temp\ipykernel_9160\4172117263.py:18: FutureWarning: The frame.append method is depreca

In [9]:
train_set

,index,date,lon,lat,fCO2,Chl,Temp,Salt,month,year
0,0,1997/9/16,165.75,18.25,-0.434411,1.032665,1.213488,1.832317,9,1997
1,4,1997/9/16,-126.25,21.25,-0.012574,1.044124,0.684669,1.451755,9,1997
2,8,1997/9/16,165.75,25.25,-0.355499,1.045368,1.143862,2.247605,9,1997
3,12,1997/9/16,164.75,22.25,-0.435739,1.048461,1.182022,2.02878,9,1997
5,20,1997/9/16,164.75,14.25,-0.232454,1.05321,1.201011,1.531342,9,1997
...,...,...,...,...,...,...,...,...,...,...
290,341052,2020/11/16,-126.25,47.25,0.571698,2.177229,-1.141629,-0.905404,11,2020
291,341056,2020/11/16,133.25,35.75,-0.219793,2.315672,0.001754,0.478105,11,2020
292,341060,2020/11/16,-130.25,50.75,0.511804,2.335595,-1.246181,-0.910569,11,2020
294,341068,2020/11/16,130.75,34.25,0.029917,2.867964,0.16045,0.842782,11,2020


In [18]:
def compute_distances(P, C):
    A = (P**2).sum(axis=1, keepdims=True)
    B = (C**2).sum(axis=1, keepdims=True).T
 
    return np.sqrt(A + B - 2* np.dot(P, C.T))

In [6]:
len(test_li)

1621

In [25]:
def process_df(my_set, varName):
    temp_df = pd.DataFrame()

    dataset = my_set.reset_index(drop=True)

    temp_df['label'] = dataset[varName[0]]
    temp_df['beta'] = np.ones(dataset.shape[0])
    temp_df[varName[1:4]] = dataset[varName[1:4]]

    cor_df = pd.DataFrame()

    dataset['lon'][dataset['lon'] < 0] = dataset['lon'][dataset['lon'] < 0].copy() + 360.0
    cor_df['xcor'] = dataset['lon']
    cor_df['ycor'] = dataset['lat']

    sample_pt = np.array([[110.0, 0.0], [290.0,0.0], [110.0, 70.0], [290.0, 70.0]])

    cor_li = cor_df.to_numpy()
    dis_li = compute_distances(cor_li, sample_pt)
    dis_df = pd.DataFrame(dis_li)
    temp_df = temp_df.join(dis_df)

    return temp_df



train_data = MYDataset(process_df(my_set=train_set, varName=varName), len(varName))
test_data = MYDataset(process_df(my_set=test_set, varName=varName), len(varName))
train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=0, drop_last=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False, num_workers=0)

D:\Temp\ipykernel_20216\440098623.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['lon'][dataset['lon'] < 0] = dataset['lon'][dataset['lon'] < 0].copy() + 360.0
D:\Temp\ipykernel_20216\440098623.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['lon'][dataset['lon'] < 0] = dataset['lon'][dataset['lon'] < 0].copy() + 360.0


In [6]:
relation = str()
relation = varName[0]+"~" + "+".join(varName[1:len(varName)])
fit=sm.formula.ols(relation,data=train_set).fit()

In [21]:
process_df(my_set=train_set, varName=varName)

D:\Temp\ipykernel_20216\911688131.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_set['lon'][my_set['lon'] < 0] = my_set['lon'][my_set['lon'] < 0] + 360.0


,label,beta,Chl,Temp,Salt,0,1,2,3
0,0.584651,1.0,1.239896,1.060443,1.955315,36.812022,154.742124,52.821634,159.311409
1,0.373193,1.0,1.251547,1.047341,1.939265,37.491666,154.905213,51.967538,159.030264
2,-0.205309,1.0,1.257564,1.129270,2.020885,34.060608,153.525649,57.403179,160.328179
3,0.373193,1.0,1.261583,1.020600,1.939265,37.856637,154.412192,52.231456,158.550071
4,0.046590,1.0,1.262256,1.167901,1.966213,31.908071,153.062487,60.935417,161.626498
...,...,...,...,...,...,...,...,...,...
6476,-2.875452,1.0,4.474066,-1.788168,-1.140323,94.337824,124.376947,72.281567,108.603062
6477,-0.061960,1.0,4.996762,0.459170,0.839635,45.101275,155.190609,45.487636,155.303332
6478,-4.757780,1.0,5.694138,-2.185876,-0.963957,101.181149,121.027373,77.927049,102.384691
6479,-1.273553,1.0,6.892250,-1.441565,0.155700,68.633993,137.915282,55.368086,131.816634


In [7]:
model = SWNN(outsize=4)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [27]:
varName[0]+"+" + "~".join(varName[1:len(varName)])

'fCO2+Chl~Temp~Salt'

In [7]:
r2 = 0
weightlist = [[]]
temp = []
for j in fit.params:
    weightlist[0].append(j)
out = nn.Linear(4, 1, bias = False)
out.weight = nn.Parameter(torch.tensor(weightlist), requires_grad=False)

def train(epoch):
    model.train()
    train_loss = 0
    global r2
    global out
    for data, coef, label in train_loader:
        data = data.view(data.shape[0], -1)
        label = label.view(data.shape[0], -1)
        optimizer.zero_grad()

        output = model(data)
        output = output.mul(coef)
        output = out(output)

        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        a = output.view(-1).detach().numpy()
        b = label.view(-1).numpy()
        if epoch % 100 == 0:
            r2 = r2_score(a, b)

        train_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(train_loader.dataset)
    print('\r Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

def val(epoch):
    model.eval()
    global out
    global r2
    val_loss = 0

    label_li = np.array([])
    out_li = np.array([])

    with torch.no_grad():
        for data, coef, label in test_loader:
            data = data.view(data.shape[0], -1)
            label = label.view(data.shape[0], -1)

            output = model(data)
            output = output.mul(coef)
            output = out(output)

            loss = criterion(output, label)

            a = output.view(-1).detach().numpy()
            b = label.view(-1).numpy()
            out_li = np.append(out_li, a)
            label_li = np.append(label_li, b)

            val_loss += loss.item()*data.size(0)
        val_loss = val_loss/len(test_loader.dataset)
        label_li = np.array(label_li).reshape(-1)
        out_li = np.array(out_li).reshape(-1)
        if epoch % 100 == 0:
            r2 = r2_score(out_li, label_li)
        print('\r Epoch: {} \tTraining Loss: {:.6f} \tR2: {:.6f}'.format(epoch, val_loss, r2))
        if epoch % 2 == 0:
            clear()
        

In [9]:
for epoch in tqdm(range(1, 1000+1)):
    train(epoch=epoch)
    val(epoch=epoch)

100%|██████████| 1000/1000 [12:31<00:00,  1.33it/s]


In [ ]:
for data, coef, label in train_loader:
    print(label.view(50,-1))